# Import Packages

In [1]:
import pandas as pd
import numpy as np
from gekko import GEKKO

# Import Data



In [2]:
forecasts = pd.read_csv('C:/dev/verteiler/02_Forecasting_Benchmark/Wareverschiebung-VZ-zu-Filiale.csv')

In [3]:
forecasts.head(5)

,Week,Year,Site_ID,Article_ID,Forecast,Min Stock,Article_Desc,Mainsector_ID,Mainsector_Desc,Warengruppe_ID,Warengruppe_Desc,Vendor_ID,Vendor_Name
0,9,2020,107,1171625400,47.25,95.0,BECHER HOBSTAR 35CL,45,Home/Lifestyle,730,Tisch Hartwaren,147322,Kreavita Home Design AG
1,9,2020,101,1261293900,44.50,89.0,GIN TONIC MIXOLOGY 80CL,45,Home/Lifestyle,730,Tisch Hartwaren,147322,Kreavita Home Design AG
2,9,2020,116,25587914,35.00,70.0,DREAMFLOR SAND 30X30,45,Home/Lifestyle,630,Bad-Textilien,114207,Weseta Textil AG
3,9,2020,101,1280659300,27.25,55.0,STABKERZE SCHWARZ 22X240MM,45,Home/Lifestyle,988,Wohnboutique,942081,Herzog Kerzen AG
4,9,2020,101,23605368,24.50,49.0,BASIC OTA KAFFEE,45,Home/Lifestyle,730,Tisch Hartwaren,331934,Porzellanmanufaktur Kahla/Thüringen


# Optimization

Optimization is based on section "Mixed Integer Nonlinear Programming" in https://gekko.readthedocs.io/en/latest/examples.html

In [4]:
supply = 16 # Verfügbare Menge

In [5]:


for article in forecasts.Article_ID.drop_duplicates():
    print(article)
    
    df = forecasts.loc[forecasts.Article_ID == article]
    
    demand = df['Min Stock'].sum()
    
    if supply >= demand:
        print('supply > demand')
        #continue
        break
    
    break

1171625400


In [6]:
c1 = 10 # Soll-Lager Filiale 1
c2 = 20 # Soll-Lager Filiale 2
#-> Sum(Soll-Lager) > Verfügbare Menge

In [7]:
m = GEKKO(remote=False) 
m.options.SOLVER=1


In [8]:
m.solver_options = ['minlp_maximum_iterations 500', \
                    # minlp iterations with integer solution
                    'minlp_max_iter_with_int_sol 10', \
                    # treat minlp as nlp
                    'minlp_as_nlp 0', \
                    # nlp sub-problem max iterations
                    'nlp_maximum_iterations 50', \
                    # 1 = depth first, 2 = breadth first
                    'minlp_branch_method 1', \
                    # maximum deviation from whole number
                    'minlp_integer_tol 0.05', \
                    # covergence tolerance
                    'minlp_gap_tol 0.01']

In [9]:

x1 = m.Var(lb=1,ub=c1, integer=True)
x2 = m.Var(lb=1,ub=c2,integer=True)

In [10]:
m.Equation(x1 + x2 <= supply)

In [11]:
m.Obj((c1/x1) + (c2/x2))

In [12]:
m.solve(disp=False)

In [13]:
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('Objective: ' + str(m.options.objfcnval))

#-> Anstatt wie gewünscht c1=10, c2=20 erhalten die Filialen 7 resp. 9 Artikel

x1: [7.0]
x2: [9.0]
Objective: 3.6507936508
